In [1]:
import pandas as pd

In [2]:
import sqlite3

<h2><center>Соединение с БД</center></h2>

In [3]:
connection = sqlite3.connect('../data/checking-logs.sqlite')

<h2><center>Запросы к БД</center></h2>

In [4]:
pd.io.sql.read_sql("select checker.uid, labname, timestamp as first_commit_ts, datetime as first_view_ts from checker \
                              left join (select uid, min(datetime) as datetime from pageviews group by uid) pv\
                              using(uid) where checker.uid like '%user_%' and checker.status like '%ready%' and checker.numTrials = 1 and \
                              checker.labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')", connection, \
                              parse_dates=['first_commit_ts', 'first_view_ts']).to_sql('datamart', connection, index=False, if_exists='replace')

In [5]:
datamart = pd.io.sql.read_sql('select * from datamart', connection, parse_dates=['first_commit_ts', 'first_view_ts'])

In [6]:
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [7]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


<h2><center>Создание Dataframes</center></h2>

In [8]:
test = datamart[datamart.first_view_ts.isna() == False].copy()

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [10]:
control = datamart[datamart.first_view_ts.isna()].copy()

In [11]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [12]:
control = control.query('first_view_ts != first_view_ts').fillna({'first_view_ts': test.first_view_ts.mean()})

In [13]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


<h2><center>Сохранение в базу данных</center></h2>

In [14]:
test.to_sql('test', connection, index=False, if_exists='replace')

In [15]:
pd.io.sql.read_sql('select * from test', connection) # тест для чек-листа

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [16]:
control.to_sql('control', connection, index=False, if_exists='replace')

In [17]:
pd.io.sql.read_sql('select * from control', connection) # тест для чек-листа

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783
3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783
4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783
...,...,...,...,...
76,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783
77,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783
78,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783
79,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783


<h2><center>Закрытие соединения с БД</center></h2>

In [18]:
connection.close()